In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Code to extract all images from rar files

In [ ]:
import os

rar_folder = '/content/drive/MyDrive/Project'
os.chdir(rar_folder)


In [ ]:
!apt-get install unrar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [ ]:
import glob

# Destination folder where files will be extracted
destination_folder = '/content/drive/MyDrive/Project/Images'

# Find all .rar files in the folder
rar_files = glob.glob('*.rar')

# Extract each .rar file
for rar_file in rar_files:
    print(f"Extracting {rar_file}...")
    os.system(f'unrar x "{rar_file}" "{destination_folder}"')
    print(f"{rar_file} extracted successfully!")


Extracting reconstructed R4M.rar...
reconstructed R4M.rar extracted successfully!


# Counting number of images in the folders

In [2]:
import os

In [3]:
# BOTTOM (Images are cross section of the bottom portion of the steel samples)

# Path to your folder (update this)
folder_path = "/content/drive/MyDrive/Project/Bottom"

# Define image extensions
image_extensions = {".tif"}

# Count images
num_images = sum(1 for file in os.listdir(folder_path) if file.lower().endswith(tuple(image_extensions)))

print(f"Number of images in folder: {num_images}")

Number of images in folder: 858


In [4]:
# MIDDLE (Images are cross section of the middle portion of the steel samples)

# Path to your folder (update this)
folder_path = "/content/drive/MyDrive/Project/Middle"

# Define image extensions
image_extensions = {".tif"}

# Count images
num_images = sum(1 for file in os.listdir(folder_path) if file.lower().endswith(tuple(image_extensions)))

print(f"Number of images in folder: {num_images}")

Number of images in folder: 842


# Copying images from Bottom and Middle to Original (Merging)

In [5]:
import shutil

# Define source and destination folders
source_folder1 = "/content/drive/MyDrive/Project/Middle"
source_folder2 = "/content/drive/MyDrive/Project/Bottom"
destination_folder = "/content/drive/MyDrive/Project/Original"

# Ensure destination folder exists
os.makedirs(destination_folder, exist_ok=True)

def copy_files(source, destination):
    for file_name in os.listdir(source):
        source_path = os.path.join(source, file_name)
        destination_path = os.path.join(destination, file_name)

        # If the file already exists, rename it to avoid overwriting
        if os.path.exists(destination_path):
            base, ext = os.path.splitext(file_name)
            counter = 1
            while os.path.exists(destination_path):
                new_file_name = f"{base}_{counter}{ext}"
                destination_path = os.path.join(destination, new_file_name)
                counter += 1

        shutil.copy2(source_path, destination_path)

# Merge folders by copying files
copy_files(source_folder1, destination_folder)
copy_files(source_folder2, destination_folder)

print("Merge complete! Files copied successfully.")

Merge complete! Files copied successfully.


In [6]:
# ORIGINAL (Merged folder containing Bottom and Middle cross section images)

# Path to your folder (update this)
folder_path = "/content/drive/MyDrive/Project/Original"

# Define image extensions
image_extensions = {".tif"}

# Count images
num_images = sum(1 for file in os.listdir(folder_path) if file.lower().endswith(tuple(image_extensions)))

print(f"Number of images in folder: {num_images}")

Number of images in folder: 1700


# Converting to binary masks using otsu thresholding

In [7]:
import cv2
import os
import numpy as np

# ------------------- Set Paths -------------------
input_folder = "/content/drive/MyDrive/Project/Original"  # Folder containing original images
output_folder = "/content/drive/MyDrive/Project/Binary"         # Folder to save binary masks

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# ------------------- Process All Images -------------------
for filename in os.listdir(input_folder):
    if filename.endswith(('.tif')):  # Process only .tif image files
        image_path = os.path.join(input_folder, filename)
        mask_path = os.path.join(output_folder, filename)

        # Load image in grayscale
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Apply Otsu's Thresholding
        _, mask = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Save binary mask
        cv2.imwrite(mask_path, mask)

        #print(f"Processed: {filename}")

print("All images processed and saved in:", output_folder)


All images processed and saved in: /content/drive/MyDrive/Porosity DL Project/Binary


In [8]:
# Binary

# Path to your folder (update this)
folder_path = "/content/drive/MyDrive/Project/Binary"

# Define image extensions
image_extensions = {".tif"}

# Count images
num_images = sum(1 for file in os.listdir(folder_path) if file.lower().endswith(tuple(image_extensions)))

print(f"Number of images in folder: {num_images}")

Number of images in folder: 1700


# U-Net Model (Defining & Compiling)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy

# ------------------- Define U-Net Model -------------------
def unet_model(input_size=(256, 256, 1)):
    inputs = Input(input_size)

    # Encoder (Downsampling Path)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = BatchNormalization()(c1)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    c1 = BatchNormalization()(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    c2 = BatchNormalization()(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = BatchNormalization()(c3)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    c3 = BatchNormalization()(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = BatchNormalization()(c4)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    c4 = BatchNormalization()(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = BatchNormalization()(c5)
    c5 = Dropout(0.5)(c5)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)
    c5 = BatchNormalization()(c5)

    # Decoder (Upsampling Path)
    u6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = BatchNormalization()(c6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)
    c6 = BatchNormalization()(c6)

    u7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = BatchNormalization()(c7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)
    c7 = BatchNormalization()(c7)

    u8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = BatchNormalization()(c8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)
    c8 = BatchNormalization()(c8)

    u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = BatchNormalization()(c9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)
    c9 = BatchNormalization()(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs, outputs)
    return model

# ------------------- Compile Model -------------------
model = unet_model()
model.compile(optimizer=Adam(learning_rate=1e-4), loss=[binary_crossentropy], metrics=['accuracy'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 1)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 256, 256, 64)   │            640 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 256, 256, 64)   │            256 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 256, 256, 64)   │         36,928 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 256, 256, 64)   │            256 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 128, 128, 64)   │              0 │ batch_normalization_1… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 128, 128, 128)  │         73,856 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 128, 128, 128)  │            512 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 128, 128, 128)  │        147,584 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 128, 128, 128)  │            512 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 64, 64, 128)    │              0 │ batch_normalization_3… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 64, 64, 256)    │        295,168 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 64, 64, 256)    │          1,024 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 64, 64, 256)    │        590,080 │ batch_normalization_4… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_5     │ (None, 64, 64, 256)    │          1,024 │ conv2d_5[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2      

 Total params: 31,054,145 (118.46 MB)

 Trainable params: 31,042,369 (118.42 MB)

 Non-trainable params: 11,776 (46.00 KB)

# Load Dataset (Images + Binary Masks)

In [ ]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split

# ------------------- Set Paths -------------------
image_folder = "/content/drive/MyDrive/Project/Original"
mask_folder = "/content/drive/MyDrive/Project/Binary"

IMG_SIZE = 256  # Resize to 256x256

# ------------------- Load Dataset -------------------
def load_data(image_folder, mask_folder):
    images, masks = [], []

    for filename in sorted(os.listdir(image_folder)):
        if filename.endswith(('.tif')):
            img_path = os.path.join(image_folder, filename)
            mask_path = os.path.join(mask_folder, filename)

            # Load and preprocess image
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = img / 255.0
            img = np.expand_dims(img, axis=-1)

            # Load and preprocess mask
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, (IMG_SIZE, IMG_SIZE))
            mask = mask / 255.0
            mask = np.expand_dims(mask, axis=-1)

            images.append(img)
            masks.append(mask)

    return np.array(images), np.array(masks)

X, Y = load_data(image_folder, mask_folder)
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

print(f"Data Loaded: {X.shape}, {Y.shape}")


✅ Data Loaded: (1700, 256, 256, 1), (1700, 256, 256, 1)


# Training U-Net Model

In [ ]:
from tensorflow.keras.metrics import MeanIoU

# Compile Model with IoU Metric
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss=binary_crossentropy,
              metrics=['accuracy', MeanIoU(num_classes=2)])

BATCH_SIZE = 16
EPOCHS = 10 # Gradually increase

history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), batch_size=BATCH_SIZE, epochs=EPOCHS)

# ------------------- Save Model -------------------
model.save("unet_microstructure_segmentation.h5")
print("Model Trained & Saved!")

Epoch 1/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 202s 1s/step - accuracy: 0.9426 - loss: 0.1427 - mean_io_u: 0.3273 - val_accuracy: 0.4050 - val_loss: 0.7107 - val_mean_io_u: 0.3273
Epoch 2/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 90s 897ms/step - accuracy: 0.9872 - loss: 0.0245 - mean_io_u: 0.3274 - val_accuracy: 0.3927 - val_loss: 0.8289 - val_mean_io_u: 0.3273
Epoch 3/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 86s 941ms/step - accuracy: 0.9876 - loss: 0.0176 - mean_io_u: 0.3274 - val_accuracy: 0.6169 - val_loss: 0.5928 - val_mean_io_u: 0.3273
Epoch 4/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 78s 893ms/step - accuracy: 0.9877 - loss: 0.0157 - mean_io_u: 0.3275 - val_accuracy: 0.9585 - val_loss: 0.1533 - val_mean_io_u: 0.3273
Epoch 5/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 82s 894ms/step - accuracy: 0.9878 - loss: 0.0148 - mean_io_u: 0.3279 - val_accuracy: 0.9751 - val_loss: 0.0656 - val_mean_io_u: 0.3273
Epoch 6/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 86s 940ms/step - accuracy: 0.9878 - loss: 0.0143 - mean_io_u: 0.3291 - val_accuracy: 0.9810 - val_l

# Evaluation

In [ ]:
# Evaluate Model
loss, accuracy, iou = model.evaluate(X_val, Y_val)
print(f"Model Performance:")
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"IoU (Jaccard Index): {iou:.4f}")  # IoU should be between 0 and 1


11/11 ━━━━━━━━━━━━━━━━━━━━ 64s 3s/step - accuracy: 0.9880 - loss: 0.0126 - mean_io_u: 0.3385
✅ Model Performance:
Loss: 0.0126
Accuracy: 0.9880
IoU (Jaccard Index): 0.3383
